In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from dateutil.relativedelta import relativedelta
import time
from datetime import datetime, timedelta

In [11]:
import requests
from bs4 import BeautifulSoup as bs

In [12]:
def get_naver_frequency(x,y):
    url = 'https://search.naver.com/search.naver?where=post&query={}&st=sim&sm=tab_opt&date_from={}&date_to={}&date_option=8&srchby=all&dup_remove=1&post_blogurl=&post_blogurl_without=&nso=so%3Ar%2Ca%3Aall%2Cp%3Afrom{}to{}'
    make_start = pd.to_datetime(str(y))
    make_end = make_start +timedelta(days=1)
    z = make_end.strftime('%Y%m%d')
    
    URL = url.format(x,y,y,y,y)
    page = requests.get(URL)
    time.sleep(1)

    soup = bs(page.text,'html.parser')
    content= soup.find('div',{'class':'blog section _blogBase _prs_blg'})
    whole = content.find('div',{'class':'section_head'})
    result = whole.find('span',{'class':'title_num'})
    
    if result is None:
        return(0)  
    else:
        found = result.text
        split = found.split('/')[1]
        maybe = split.replace("약","")
        number = maybe.replace("건","")
        nocom = number.replace(",","")
        num = int(nocom)
        return(num)

In [13]:
def get_date_in_line(year,month,day,x):
    start_date = date(year,month,day)
    day_list = []
    for how_many in range(1,x+1):
        formattedDate = start_date.strftime("%Y%m%d")
        day_list.append(formattedDate)
        start_date += timedelta(days=1)
    return(day_list)

In [5]:
#크롤링 시작 날짜를 년,월,일 형태로 입력합니다
#네번째 숫자는 크롤링 기간을 의미합니다 Ex. 600 = 시작일로부터 600일동안의 빈도수 크롤링
start_list = get_date_in_line(2020,5,1,70)

In [6]:
naver_frequency = []

for day in start_list:
    start = int(day)
    naver_frequency.append(get_naver_frequency('앵그리너구리',start))

In [7]:
naver = pd.DataFrame({'date':start_list,'frequency':naver_frequency})

In [ ]:
naver.to_csv('앵그리너구리출시이후.csv') 

#구글코랩 사용시 코드
from google.colab import files
naver.to_csv('채황라면출시이후.csv') 
files.download('채황라면출시이후.csv')

In [8]:
#일자별로 게시글 빈도수를 나타내는 csv
naver.to_csv(r'앵그리너구리출시이후.csv', index = False)

In [ ]:
#년월일로 표시된 date칼럼에서 연도와 월 정보만 추출하는 부분입니다
date_list = list(naver['date'])

new_list = []
for item in date_list:
    itemstr = str(item)
    new_list.append(itemstr[0:6])

naver['date'] = new_list

In [ ]:
#월을 기준으로 데이터를 추합하는 부분입니다
new_naver = naver.groupby(['date']).sum()
rearr_X = new_naver.reset_index()

In [ ]:
#월별로 게시글 빈도수를 나타내는 csv
rearr_X.to_csv(r'파일위치\파일명.csv', index = False)